### This is the second Part of Char level Modeling and will discuss One Layer Neural Model for same
### Complete First Part before starting this
By Naveen Aggarwal

#### First Step in developing a NN Model is to create a training set. In this case, all bigrams will be part of training set

In [1]:
import torch

In [36]:
# Indian Names: NamesIndian.txt, Foreign Names: names.txt
words=open("NamesIndian.txt", 'r').read().splitlines()
# If we read indian names, Then we have to do some extra work in next cell

In [44]:
# keep everything in lower case, strip blank spacesm replace - and ''
import re
print(words[:10])
a = (map(lambda x: x.lower().strip().replace('-','',1).replace(' ','',1), words))
#a_new=re.sub('[ -]','',a)
words = list(a)
print(words[:10])

['aaban', 'aabharan', 'aabhas', 'aabhat', 'aabheer', 'aabheer', 'abheer', 'aabher', 'aabi', 'aabilesh']
['aaban', 'aabharan', 'aabhas', 'aabhat', 'aabheer', 'aabheer', 'abheer', 'aabher', 'aabi', 'aabilesh']


In [46]:
#chars= list('.ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz')
#start_token='.'
# Char list created in this way. As extra character was creating problem.

chars= sorted(set("".join([start_token]+ words )))
print(chars)
stoi={s:i for i,s in enumerate(chars)}
#stoi

# Lets also create itos, we will use later
itos={i:s for i,s in enumerate (chars)}
#itos

[' ', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [47]:
# Create a Training set and run for first word to test the code
xs, ys = [], []
for w in words [:1]:
    ws=  ["."] + list(w) + ["."]
    for ch1, ch2 in zip (ws[:], ws[1:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        print (ch1, ch2)
        xs.append(ix1)
        ys.append(ix2)
# Convert into tensors

xs=torch.tensor(xs)
ys=torch.tensor(ys)
print(xs, ys)

# You can check how traing set is created, For 0 value in xs, it is 5 in ys. For 5 value in xs, it is 13 in ys. 
# Similar to bigrams are read

. a
a a
a b
b a
a n
n .
tensor([ 1,  2,  2,  3,  2, 15]) tensor([ 2,  2,  3,  2, 15,  1])


### Now to use this into the NN Model, We have to convert these integers into one hot encoding

In [48]:
# We will use one hot encoding function of torch
import torch.nn.functional as f

# num_classes is important to provide, otherwise it will take largest number as "No of classes". 
#In Neural network, we want to use Float values

xhot=f.one_hot(xs, num_classes=28).float()
#xhot

#### Now Initalize the weights

In [49]:
# Number of classes depends upon total character in the matrix. Due to extra character, Number of classes are 28
W=torch.randn((28,28))

#### If we use generator, then we can generate same random numbers again. This is helpful, when you are developing and testing 

In [50]:
g=torch.Generator().manual_seed(214783647)
W=torch.randn((28,28), generator=g, requires_grad=True)

In [51]:
# Lets See the size of xhot and W
print (xhot.shape, W.shape)

torch.Size([6, 28]) torch.Size([28, 28])


In [52]:
# Now Lets multiply, In torch use @ for multiplication
logit=xhot@W
logit.shape

torch.Size([6, 28])

In [53]:
# We will consider these as log count, So we will take exp. exp will be 0 to 1for negative numbers and greater than  1 for positive numbers
counts=logit.exp()
# This count we will take as equivalent to N
# Now Probability is just a normalized count
probs=counts /counts.sum(1, keepdims=True)
print (probs.shape)
print (probs[0].sum())

torch.Size([6, 28])
tensor(1.0000, grad_fn=<SumBackward0>)


#### Now Lets combine above all steps and run it on all words, We will call it a forward pass

In [54]:
# Forward Pass
xhot=f.one_hot(xs, num_classes=28).float()
logit=xhot@W
counts=logit.exp()
probs=counts /counts.sum(1, keepdims=True)
# We will define loss function as difference from actual value in ys
loss= -probs[torch.arange(6),ys].log().mean()

In [55]:
print(loss.item())

4.243195533752441


#### Now lets write code for Backward Pass. In backward pass, we have to calculate the gradients and update the weights
#### Since Pytorch makes the computation graph. If computations are differentiable, Then Pytorch calculate the back ward pass automatically. We have to just call the backward function

In [56]:
# Backward Pass
W.grad=None
loss.backward()

#Update
W.data+=-(0.1)*W.grad

#### Now Lets execute these in loop for all words

In [57]:
# create Dataset
xs, ys = [], []
for w in words:
    ws=  ws=["."] + list(w) + ["."]
    for ch1, ch2 in zip (ws[:], ws[1:]):
        ix1=stoi[ch1]
        ix2=stoi[ch2]
        #print (ch1, ch2)
        xs.append(ix1)
        ys.append(ix2)
# Convert into tensors

xs=torch.tensor(xs)
ys=torch.tensor(ys)
num=xs.nelement()
print(num)

# initialize network
g=torch.Generator().manual_seed(214783647)
W=torch.randn((28,28), generator=g, requires_grad=True)

505470


In [59]:
for k in range(100):
    # Forward Pass
    xhot=f.one_hot(xs, num_classes=28).float()
    logit=xhot@W
    counts=logit.exp()
    probs=counts /counts.sum(1, keepdims=True)
    # We will define loss function as difference from actual value in ys
    loss= -probs[torch.arange(num),ys].log().mean()
    print(loss.item())
    
    # Backward Pass
    W.grad=None
    loss.backward()

    #Update
    #W.data+=-(0.1)*W.grad
    W.data+=-(50)*W.grad
    
    
    

3.890211343765259
3.25390362739563
2.918787717819214
2.7444732189178467
2.63631534576416
2.5683062076568604
2.515385627746582
2.4837374687194824
2.451887845993042
2.434147357940674
2.410933494567871
2.3999452590942383
2.3812429904937744
2.374174118041992
2.358194351196289
2.3536746501922607
2.3395702838897705
2.336874485015869
2.3241775035858154
2.3228671550750732
2.3112704753875732
2.311048746109009
2.3003318309783936
2.300985813140869
2.290984630584717
2.292355537414551
2.2829442024230957
2.284910202026367
2.275989294052124
2.278453826904297
2.26994252204895
2.2728261947631836
2.2646591663360596
2.2678956985473633
2.260016918182373
2.263551950454712
2.255915880203247
2.2597029209136963
2.2522706985473633
2.256272315979004
2.2490122318267822
2.2531964778900146
2.246082305908203
2.2504234313964844
2.2434332370758057
2.2479095458984375
2.2410264015197754
2.2456204891204834
2.2388291358947754
2.243525981903076
2.2368149757385254
2.2416021823883057
2.23496150970459
2.2398293018341064
2.23

#### Now lets generate few names from it

In [62]:
for i in range(10):
    out=[]
    ix=0
    while True:
        # Earlier we did this in previous Notebook
        #p=P[ix]
        # But now we will do this
        
        xhot=f.one_hot(torch.tensor([ix]), num_classes=28).float()
        logit=xhot@W
        counts=logit.exp()
        p=counts /counts.sum(1, keepdims=True)

        ix=torch.multinomial(p, num_samples=1, replacement= True, generator=g).item()
        out.append(itos[ix])
        if ix == 1:
            break
    print(''.join(out))

umala.
chan.
lan.
qbjrikrujthuthi.
oradan.
q varanthinikan.
chujathandaneshekand.
ilxdhujkaneimhajeralvanak.
chanshapi.
mila.
